In [0]:
%sql

select * from dimruncycle
order by runcycleid desc
limit 1;

-- DROP TABLE IF EXISTS dimrequeststatuses;

-- CREATE TABLE dimrequeststatuses
-- USING DELTA AS
--     SELECT
--         requeststatusid,
--         name AS requeststatusname,
--         name AS requeststatus,
--         CAST(NULL AS TIMESTAMP) AS createddate,
--         CAST(NULL AS TIMESTAMP) AS modifieddate,
--         CASE
--             WHEN isactive = true THEN 'Y'
--         ELSE
--             'N'
--         END AS cactive,
--         'FOIMOD' AS sourceoftruth
--     FROM foi_mod.foirequeststatuses
--     order by requeststatusid;

In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
from pyspark.sql.functions import col #lit

tablename = "dimrequeststatuses"
runcycleid = etl_helpers.start_run_cycle(tablename)
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    query = f"""
        SELECT
            requeststatusid,
            name AS requeststatusname,
            name AS requeststatus,
            CAST(NULL AS TIMESTAMP) AS createddate,
            CAST(NULL AS TIMESTAMP) AS modifieddate,
            CASE
                WHEN isactive = true THEN 'Y'
            ELSE
                'N'
            END AS cactive,
            'FOIMOD' AS sourceoftruth
        FROM foi_mod.foirequeststatuses
        order by requeststatusid;
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    # order of columns here is important!
    df_mapped = df.selectExpr(
            "requeststatusid AS requeststatusid",
            "requeststatusname AS requeststatusname",
            "requeststatus AS requeststatus",
            "createddate AS createddate",
            "modifieddate AS modifieddate",
            "cactive AS cactive",
            "sourceoftruth AS sourceoftruth"
        )
    df_mapped.show()

    from delta.tables import DeltaTable
    delta_table = DeltaTable.forName(spark, f"hive_metastore.default.{tablename}")
    delta_table.alias("target").merge(
        df_mapped.alias("source"),
        "target.requeststatusid = source.requeststatusid AND target.sourceoftruth = source.sourceoftruth"
    ).whenMatchedUpdate(set = {
        "requeststatusname": col("source.requeststatusname"),
        "requeststatus": col("source.requeststatus"),
        "createddate": col("source.createddate"),
        "modifieddate": col("source.modifieddate"),
        "cactive": col("source.cactive")
    }).whenNotMatchedInsert(values = {
        "requeststatusid": col("source.requeststatusid"),
        "requeststatusname": col("source.requeststatusname"),
        "requeststatus": col("source.requeststatus"),
        "createddate": col("source.createddate"),
        "modifieddate": col("source.modifieddate"),
        "cactive": col("source.cactive"),
        "sourceoftruth": col("source.sourceoftruth")
    }).execute()

    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
    dbutils.notebook.exit("Error: Something went wrong.")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")
        dbutils.notebook.exit("Error: Something went wrong.")